In [1]:
%pip install youtube-transcript-api tiktoken faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [2]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [22]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel

In [40]:
youtube_id="iOdFUJiB0Zc"
transcript_list=YouTubeTranscriptApi.get_transcript(youtube_id,languages=["en"])

In [41]:
transcript=" ".join(docs['text'] for docs in transcript_list)

In [42]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=splitter.create_documents([transcript])

In [43]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
vector_store=FAISS.from_documents(documents,OpenAIEmbeddings())

In [44]:
vector_store.index_to_docstore_id

{0: '008eda62-6389-4bd3-a434-292cddecd2f6',
 1: '735aff10-ed87-4a44-9cec-d64cc0042d96',
 2: '78921018-9360-4336-90d2-3ff987a1d1a0',
 3: 'bc219359-cb77-4c83-a291-e1787d480e0c',
 4: '9458b7be-84f6-4399-a3c2-7c687bc7968e',
 5: '1152ccb5-f1b7-4297-b0d7-05e0119da4a2',
 6: 'bdd044d7-2038-4a07-8c99-724f730945a0',
 7: 'c1824b8b-75de-4daa-9033-ca9c60550879',
 8: 'b12553d5-63d8-44a7-b236-5fd1cc52a7af',
 9: 'fa95994d-5f13-47eb-b491-0a91f1888ba9',
 10: '8bc5b093-92c6-4a9e-a4f2-3d6aade2c2c7',
 11: '08f3f258-1a46-4ad7-9128-88e994f5398b',
 12: '58656d7e-7090-494d-90f3-049a621ddc9b',
 13: '10caf6a0-d9c3-4aa5-9893-579f6e2bca72',
 14: 'b9688920-4955-4236-a678-5cc079baab80',
 15: '62fa98f8-de5a-4e4d-9fd8-70bd446893b8',
 16: '54b50b6f-eac4-46db-b8ee-b05052422f23',
 17: '9e672d95-cd5b-44fe-985d-0709c81079e8',
 18: 'f9c0f767-2746-41e0-ab86-978d065ad584',
 19: '39c5596f-a3f5-4093-a607-65f5fbb35762',
 20: '8cdf52ba-f724-469d-87ba-2699d8e6a4b0',
 21: '56961906-e05a-45f0-ac58-0d40abf151c5',
 22: 'e24b50be-6e50-

In [46]:
vector_store.get_by_ids(['2176b397-37ec-4e36-9887-8492ec04ac13'])[0].page_content

"train it and hardly you know just to put my input data it hardly took very less time right so this is one amazing application this is how you can probably do the fine tuning one task I really want to give it to you I okay let's let's reload this I think you'll be able to see my fine tuned model also uh let's see the model uh okay I've already deleted it I think yeah I've deleted it so that is the reason you're not able to see it over here if I had not deleted that you could also see the models over here right so model testing see model is also not here right otherwise you could also see it uh put it over here and you can ask any question you want right this is quite amazing so at the end of the day if you don't want to delete it just don't delete it and keep it like this so I hope you like this particular video I hope you have understood how you can fine tune our amazing llm models uh in with the help of gradient AI Cloud uh which is quite amazing you should definitely use it try to"

In [47]:
retriever=vector_store.as_retriever(search_type='similarity',search_kwargs={"k":4})

In [48]:
retriever.invoke("what is ai")

[Document(id='0f241f34-c84d-4bfa-9921-95a3ec1e8136', metadata={}, page_content="logged in and then we will go ahead and see to it let's see I will go ahead and log in in this way so once we login over here so here you can see that you'll be able to see a dashboard okay once you probably log in now after you log in all you have to do is that go and towards this AI project section okay if you have not logged in you have to sign up in the specific platform now what I will do is that I will go ahead and create a AI project so let me go ahead and create an AI project and I will name this AI project to something else let's say I will write uh document Q&A something I can write up to you okay so whatever application let's say I will say rag system right so I I'm actually trying to create this I will enable this also make sure that you enable this then only your application will probably get started now in this entire pipeline initially you'll be able to see two important section one is query 

In [50]:
def form_retrieved_docs(retrieved_docs):
    return "\n\n".join(doc.page_content for doc in retrieved_docs)

In [51]:
parallel_chain=RunnableParallel({
    "context":retriever|RunnableLambda(form_retrieved_docs),
    "question":RunnablePassthrough()
})

In [52]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
llm=ChatOpenAI()
parser=StrOutputParser()
template=PromptTemplate(
    template="""Act as a helpful assistant. You have to give answer of question from the following context.
    {context}
    Question: {question}
    If the context is insufficient to answer. Just say you don't know.
    """,
    input_variables=["context","question"]
)

In [53]:
chain=parallel_chain|template|llm|parser

In [59]:
chain.invoke("What are the main points of this video")

'The main points of this video are discussing the concept of 1 bit LLM models, the advantages of 1 bit LLM models compared to 32 bit or 16 bit LLM models, the process of quantization, and the importance of understanding research papers and the steps involved in fine-tuning models using gradient AI Cloud.'